In [1]:
import pandas as pd
import numpy as np
import time
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import datetime
import itertools
from time import sleep

In [2]:
chrome_options = Options()
chrome_options.headless = False
driver = webdriver.Chrome(options=chrome_options)

In [3]:
Airlines = []
Aeroplane = []
Dept_time = []
Arrival_time = []
Source = []
Destination = []
Duration = []
Stops = []
Date = []
Price = []
Free_Meal = []
Checkin_Baggage = []
Taxes = []
Day_Track = []


In [4]:
start_date = datetime.date(2023,12,25)
end_date = datetime.date(2024,1,9)
delta = datetime.timedelta(days=1)
dates = [start_date+delta*i for i in range((end_date-start_date).days+1)]
cities = ["Mumbai","New", "Benga", "Kolk", "Chennai"]
routes = list(itertools.permutations(cities,2))

for date in dates:
    for route in routes:
        driver = webdriver.Chrome(options=chrome_options)
        dte = date.strftime("%d/%m/%Y")
        departure_city, arrival_city = route  # Unpack the tuple

        url = "https://www.yatra.com/"
        driver.get(url)
        time.sleep(2)
        driver.maximize_window()
        from selenium.webdriver.common.keys import Keys
        wait = WebDriverWait(driver, 8)
        time.sleep(5)

        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "webklipper-publisher-widget-container-notification-frame"))
        )
        driver.switch_to.frame(iframe)

        # Now that we are in the iframe, locate the close button and click it
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "close"))
        )
        close_button.click()

        # Switch back to the main window
        driver.switch_to.default_content()


        # driver.execute_script("arguments[0].click();", ad_close_button)
        depart_from = driver.find_element(By.XPATH, "//input[@id='BE_flight_origin_city']")
        depart_from.click()
        time.sleep(2)
        depart_from.send_keys(departure_city)
        time.sleep(2)
        depart_from.send_keys(Keys.ENTER)
        time.sleep(4)
        going_to = driver.find_element(By.XPATH, "//input[@id='BE_flight_arrival_city']")
        going_to.send_keys(arrival_city)
        going_to.click()
        time.sleep(2)
        going_to.send_keys(Keys.ENTER)
        time.sleep(4)
        wait = WebDriverWait(driver, 5)
        try:
            origin = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='BE_flight_origin_date']")))
            origin.click()

            all_dates = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='monthWrapper']//tbody//td[not(contains(@class, 'inActiveTD'))]")))
            for ddate in all_dates:
                if ddate.get_attribute("data-date") == dte:  # Verify the correct attribute
                    driver.execute_script("arguments[0].scrollIntoView(true);", ddate)
                    ddate.click()
                    break
        except TimeoutException:
            print("Element not found or not clickable.")

        # Code to click on the button with the ID 'BE_flight_fsearch_btn'
        search_button = driver.find_element(By.ID, "BE_flight_flsearch_btn")
        search_button.click()
        time.sleep(8)
        from tqdm import tqdm
        for _ in tqdm (range(0,50)):
            time.sleep(0.1)
            driver.execute_script("window.scrollBy(0,2000)","")
            time.sleep(0.2)
            driver.execute_script("window.scrollBy(0,-500)","")
        flights=driver.find_elements(By.XPATH,'//div[@class="flight-det table full-width clearfix"]')
        len(flights)

        for flight in flights:
            Date.append(date.strftime("%d/%m/%Y"))

        # wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')))

        # # Click on the "View Details" button to reveal additional information
        # view_details_buttons = driver.find_elements(By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')

        # # Iterate over each button and click it
        # for button in view_details_buttons:
        #     driver.execute_script("arguments[0].click();", button)
        #     # Wait for the additional details of each flight to be displayed
        #     wait.until(EC.visibility_of_element_located((By.XPATH, '//div[contains(@class, "flight-details")]')))

        # Extracting Airlines company name
        airlines = driver.find_elements(By.XPATH,"//div[@class='fs-13 airline-name no-pad col-8']/span")
        for i in airlines:
            Airlines.append(i.text.replace("\n"," "))
        else :
            pass

        # Extracting model of Aeroplane
        aeroplanes = driver.find_elements(By.XPATH,'//p[@class="normal fs-11 font-lightestgrey no-wrap fl-no"]/span')
        for i in tqdm(aeroplanes):
            Aeroplane.append(i.text) 
        else:
            pass

        # Extracting Departure Time
        dept_times = driver.find_elements(By.XPATH,'//div[@class="i-b pr"]')
        for i in tqdm(dept_times):
            Dept_time.append(i.text)
        else :
            pass

        # Extracting Arrival Time
        arrival = driver.find_elements(By.XPATH,'//p[@class="bold fs-15 mb-2 pr time"]')
        for i in tqdm(arrival):
            Arrival_time.append(i.text)
        else :
            pass

        # Extracting Departure city
        source = driver.find_elements(By.XPATH,"//div[@class='i-b col-4 no-wrap text-right dtime col-3']/p")
        for i in tqdm(source):
            Source.append(i.text)
        else :
            pass

        # Extracting Arrival city
        destination = driver.find_elements(By.XPATH,'//div[@class="i-b pdd-0 text-left atime col-5"]/p[2]')
        for i in tqdm(destination):
            Destination.append(i.text)
        else:
            pass

        # Extracting Duration of Flight
        duration = driver.find_elements(By.XPATH,'//div[@class="stop-cont pl-13"]/p')
        for i in tqdm(duration):
            Duration.append(i.text)
        else:
            pass

        # Extracting layovers
        stop = driver.find_elements(By.XPATH,"//div[@class='stop-cont pl-13']/div/span")
        for i in tqdm(stop):
            Stops.append(i.text)
        else:
            pass

        # Extracting Price
        count = 0
        price = driver.find_elements(By.XPATH,'//div[@class="i-b tipsy fare-summary-tooltip fs-18"]')
        for i in tqdm(price):
            Price.append(i.text.replace(',',''))
            count = count +1
        else:
            count = count +1
            pass
        
        wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')))

        # Click on the "View Details" button to reveal additional information
        view_details_buttons = driver.find_elements(By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')

        # Iterate over each button and click it
        for button in view_details_buttons:
            driver.execute_script("arguments[0].click();", button)
            # Wait for the additional details of each flight to be displayed
            wait.until(EC.visibility_of_element_located((By.XPATH, '//div[contains(@class, "flight-details")]')))


        flight_blocks = driver.find_elements(By.XPATH, '//div[@class="flightItem border-shadow pr"]')
        for block in flight_blocks:
            meal_element = block.find_elements(By.XPATH, './/span[@class="meal"]')
            if meal_element:
                meal_text = meal_element[0].text 
                Free_Meal.append(meal_text)
            else:
                Free_Meal.append("Paid Meal")


        for block in flight_blocks:
            checkin_baggage = block.find_elements(By.XPATH, './/span[@class="font-lightgrey"]')
            if checkin_baggage:
                checkin_baggage_text = checkin_baggage[0].text  
                Checkin_Baggage.append(checkin_baggage_text) 
            else:
                Checkin_Baggage.append("0 kgs")


        ext_blocks = driver.find_elements(By.XPATH, '//div[@class="flight-details pr slide-down ovf-hidden full-width table"]')
        for block in ext_blocks:
            tax_el = block.find_elements(By.XPATH, './/td[@class="text-right"]')
            if tax_el:
                tax_el_text = tax_el[1].text 
                Taxes.append(tax_el_text) 
            else:
                
                Taxes.append("0")


        for block in flight_blocks:
            day = block.find_elements(By.XPATH, './/div[@class="bold fs-12 mb-5"]')
            if day:
                day_text = day[0].text
                Day_Track.append(day_text)
            else:
                Day_Track.append("0")

        driver.quit()


100%|██████████| 85/85 [00:01<00:00, 63.20it/s]


In [ ]:
print(len(Airlines))
print(len(Aeroplane))
print(len(Dept_time))
print(len(Arrival_time))
print(len(Source))
print(len(Destination))
print(len(Duration))
print(len(Stops))
print(len(Date))
print(len(Price))
print(len(Free_Meal))
print(len(Checkin_Baggage))
print(len(Taxes))
print(len(Day_Track))

3352
3352
3352
3352
3352
3352
3352
3352
3352
3352
3352
3352
3352
3352


In [ ]:
Flight = pd.DataFrame({
    "Airline": Airlines,
    'Aeroplane': Aeroplane,
    "Date": Date,
    "Departure_Time": Dept_time,
    "Arrival_Time": Arrival_time,
    "Source": Source,
    "Destination": Destination,
    "Stops": Stops,
    "Duration": Duration,
    "Price": Price,
    "Meal Option": Free_Meal,
    "Checkin Baggage Allowed": Checkin_Baggage,
    "Taxes": Taxes,
    "Day": Day_Track
})

Flight.tail(20)

,Airline,Aeroplane,Date,Departure_Time,Arrival_Time,Source,Destination,Stops,Duration,Price,Meal Option,Checkin Baggage Allowed,Taxes,Day
3332,Vistara,UK-826/771,28/02/2024,14:30,09:40\n+ 1 day,Chennai,Kolkata,1 Stop,19h 10m,11491,Free Meal,1 piece,"1,367","Wed, 28 Feb 2024"
3333,Vistara,UK-822/773,28/02/2024,09:40,16:50,Chennai,Kolkata,1 Stop,7h 10m,11991,Free Meal,1 piece,"1,391","Wed, 28 Feb 2024"
3334,Vistara,UK-828/773,28/02/2024,07:00,16:50,Chennai,Kolkata,1 Stop,9h 50m,11991,Free Meal,1 piece,"1,391","Wed, 28 Feb 2024"
3335,Vistara,UK-824/773,28/02/2024,20:30,16:50\n+ 1 day,Chennai,Kolkata,1 Stop,20h 20m,11991,Free Meal,1 piece,"1,391","Wed, 28 Feb 2024"
3336,Vistara,UK-838/737,28/02/2024,21:10,18:05\n+ 1 day,Chennai,Kolkata,1 Stop,20h 55m,12451,Free Meal,1 piece,"1,412","Wed, 28 Feb 2024"
3337,Vistara,UK-832/737,28/02/2024,06:50,18:05,Chennai,Kolkata,1 Stop,11h 15m,12472,Free Meal,1 piece,"1,413","Wed, 28 Feb 2024"
3338,Air India,AI-440/762,28/02/2024,06:00,23:25,Chennai,Kolkata,1 Stop,17h 25m,12985,Free Meal,20 kgs,"1,525","Wed, 28 Feb 2024"
3339,Vistara,UK-826/773,28/02/2024,14:30,16:50\n+ 1 day,Chennai,Kolkata,1 Stop,26h 20m,13415,Free Meal,1 piece,"1,470","Wed, 28 Feb 2024"
3340,Air India,AI-440/764,28/02/2024,06:00,19:05,Chennai,Kolkata,1 Stop,13h 05m,13510,Free Meal,20 kgs,"1,550","Wed, 28 Feb 2024"
3341,Air India,AI-440/401,28/02/2024,06:00,08:55\n+ 1 day,Chennai,Kolkata,1 Stop,26h 55m,13510,Free Meal,20 kgs,"1,550","Wed, 28 Feb 2024"


In [ ]:
# Flight.to_csv('27_28_feb.csv')